# Analyse Exploratoire et Nettoyage des Données - Optimisation Centres de Services Publics Togo

**Objectif :** Comprendre la structure, la qualité et la distribution des données pour identifier tendances, anomalies et préparer le nettoyage.

**Auteur :** [Ton Nom]  
**Date :** 18 janvier 2026  

**Étapes EDA :**
1. Imports et configuration
2. Chargement des données
3. Description générale des jeux de données (dimensions, types de variables)
4. Analyse des valeurs manquantes, aberrantes et doublons
5. Analyse des distributions (volumes de demandes, délais, capacités)
6. Tendances spatiales et temporelles
7. Synthèse des constats

**Technologies :** Python + pandas + plotly.express + missingno (pour visualiser NaN).

In [ ]:
# Imports et configuration
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import missingno as msno
import warnings
warnings.filterwarnings('ignore')

# Configuration pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)

print("Imports réussis. Versions :")
print(f"Pandas: {pd.__version__}")
print(f"Plotly: {px.__version__}")

## 2. Chargement des Données

Chargeons les 8 fichiers CSV disponibles. Utilisons try/except pour gérer les erreurs de chargement. Encodage UTF-8 pour les accents français.

In [ ]:
# Chargement des données
data_files = {
    'demandes': '../data/demandes_service_public.csv',
    'documents': '../data/documents_administratifs_ext.csv',
    'centres': '../data/centres_service.csv',
    'communes': '../data/details_communes.csv',
    'socioeco': '../data/donnees_socioeconomiques.csv',
    'logs': '../data/logs_activite.csv',
    'developpement': '../data/developpement.csv',
    'reseau': '../data/reseau_routier_togo_ext.csv'
}

datasets = {}
for name, path in data_files.items():
    try:
        datasets[name] = pd.read_csv(path, encoding='utf-8')
        print(f"✓ {name}: {datasets[name].shape[0]} lignes, {datasets[name].shape[1]} colonnes")
    except Exception as e:
        print(f"✗ Erreur chargement {name}: {e}")

print("\nDatasets chargés avec succès.")

## 3. Description Générale des Jeux de Données

Pour chaque dataset, affichons :
- Dimensions (lignes x colonnes)
- Types de variables (dtypes)
- Statistiques descriptives de base (pour numériques)
- Aperçu des premières lignes

Cela nous permet de comprendre la structure et identifier les variables clés (ex. : dates, géoloc, volumes).

In [ ]:
# Description des datasets
for name, df in datasets.items():
    print(f"\n=== {name.upper()} ===")
    print(f"Dimensions: {df.shape}")
    print("\nTypes de variables:")
    print(df.dtypes)
    print("\nAperçu (5 premières lignes):")
    display(df.head())
    print("\nStatistiques descriptives (numériques):")
    display(df.describe())
    print("-" * 50)

## 4. Analyse des Valeurs Manquantes, Aberrantes et Doublons

- **Valeurs manquantes :** Pourcentage par colonne, visualisation avec missingno.
- **Doublons :** Nombre de lignes dupliquées.
- **Valeurs aberrantes :** Pour numériques, utiliser IQR (Q3 - Q1) pour détecter outliers (> Q3 + 1.5*IQR).

Cela identifie les problèmes de qualité à corriger lors du nettoyage.

In [ ]:
# Fonction pour détecter outliers (IQR)
def detect_outliers_iqr(df, column):
    if df[column].dtype in ['int64', 'float64']:
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
        return len(outliers), lower_bound, upper_bound
    return 0, None, None

# Analyse par dataset
for name, df in datasets.items():
    print(f"\n=== {name.upper()} ===")
    
    # Valeurs manquantes
    missing = df.isnull().sum()
    missing_pct = (missing / len(df)) * 100
    print("Valeurs manquantes (%):")
    print(missing_pct[missing_pct > 0])
    
    # Doublons
    duplicates = df.duplicated().sum()
    print(f"Doublons: {duplicates}")
    
    # Outliers pour colonnes numériques
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        count_out, lb, ub = detect_outliers_iqr(df, col)
        if count_out > 0:
            print(f"Outliers {col}: {count_out} (limites: {lb:.2f} - {ub:.2f})")
    
    # Visualisation NaN (si missingno installé)
    if missing.sum() > 0:
        msno.matrix(df)
        print("Visualisation des NaN affichée.")
    
    print("-" * 50)

## 5. Analyse des Distributions

Focus sur les variables clés :
- **Volumes de demandes** (par type document, commune, centre).
- **Délais** (temps d'attente, traitement).
- **Capacités** (des centres).

Utilisons histogrammes, boxplots et bar charts pour identifier asymétries, modes, etc.

In [ ]:
# Distributions - Volumes de demandes
if 'demandes' in datasets:
    df_dem = datasets['demandes']
    if 'type_document' in df_dem.columns:
        fig = px.histogram(df_dem, x='type_document', title='Distribution des Types de Documents Demandés')
        fig.show()
    
    if 'nb_demandes' in df_dem.columns:
        fig = px.box(df_dem, y='nb_demandes', title='Boxplot des Volumes de Demandes')
        fig.show()

# Distributions - Délais
if 'logs' in datasets:
    df_logs = datasets['logs']
    if 'delai_traitement' in df_logs.columns:
        fig = px.histogram(df_logs, x='delai_traitement', title='Distribution des Délais de Traitement')
        fig.show()

# Distributions - Capacités
if 'centres' in datasets:
    df_cent = datasets['centres']
    if 'capacite' in df_cent.columns:
        fig = px.bar(df_cent, x='nom_centre', y='capacite', title='Capacités des Centres')
        fig.show()

## 6. Tendances Spatiales et Temporelles

- **Spatiales :** Cartes choroplèthe des demandes/capacité par commune (utilisant lat/lon de details_communes).
- **Temporelles :** Évolution des demandes/délais dans le temps (si dates présentes).

Cela révèle les zones sous-desservies et les pics saisonniers.

In [ ]:
# Tendances spatiales - Carte des demandes par commune
if 'communes' in datasets and 'demandes' in datasets:
    df_comm = datasets['communes']
    df_dem = datasets['demandes']
    # Supposons jointure sur commune
    merged = pd.merge(df_dem, df_comm, on='commune', how='left')
    if 'latitude' in merged.columns and 'longitude' in merged.columns and 'nb_demandes' in merged.columns:
        fig = px.scatter_mapbox(merged, lat='latitude', lon='longitude', size='nb_demandes', color='region',
                                title='Carte des Demandes par Commune')
        fig.update_layout(mapbox_style="open-street-map")
        fig.show()

# Tendances temporelles - Évolution des demandes
if 'logs' in datasets:
    df_logs = datasets['logs']
    if 'date' in df_logs.columns and 'nb_demandes' in df_logs.columns:
        df_logs['date'] = pd.to_datetime(df_logs['date'])
        df_agg = df_logs.groupby('date')['nb_demandes'].sum().reset_index()
        fig = px.line(df_agg, x='date', y='nb_demandes', title='Évolution Temporelle des Demandes')
        fig.show()

## 7. Synthèse des Constat

**Résumé des principales observations :**
- [À remplir après exécution : ex. "Les communes rurales ont X% de NaN sur les capacités, indiquant un besoin de données complémentaires."]
- Tendances clés : [ex. "Pics de demandes en saison Y, zones sous-desservies au nord."]
- Prochaines étapes : Nettoyage basé sur ces insights (imputation, exclusions).

**Export des visualisations :** Sauvegarder les graphs en PNG pour le rapport.